#### Import everthing needed

In [1]:
#import tensorflow and sklearn stuff
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf

import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_curve, auc
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import *
#import models
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers

#### function for training in batches

In [2]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train+val', 'train')
     valid_path = os.path.join(base_dir, 'train+val', 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(zca_whitening=False,
                             horizontal_flip=True,
                             vertical_flip=True)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

#### define image size

In [3]:
IMAGE_SIZE = 96

input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)

model_input = Input(input_shape)

## Load in models


In [4]:
weights_Efficient = r'C:\Users\20192653\OneDrive - TU Eindhoven\Year 3\Q3\8P361 - Project imaging\Github\8p361group10\Final Project\EfficientNet_weights.hdf5'
weights_Xception = r'C:\Users\20192653\OneDrive - TU Eindhoven\Year 3\Q3\8P361 - Project imaging\Github\8p361group10\Final Project\Xception_weights.hdf5'
weights_Inception = r'C:\Users\20192653\OneDrive - TU Eindhoven\Year 3\Q3\8P361 - Project imaging\Github\8p361group10\Final Project\Inception_weights.hdf5'
weights_ResNet =r'C:\Users\20192653\OneDrive - TU Eindhoven\Year 3\Q3\8P361 - Project imaging\Github\8p361group10\Final Project\ResNet_weights.hdf5'
weights_MobileNetV2 = r'C:\Users\20192653\OneDrive - TU Eindhoven\Year 3\Q3\8P361 - Project imaging\Github\8p361group10\Final Project\MobileNetV2_weights.hdf5'

In [ ]:
model_Efficient =tf.keras.models.load_model(weights_Efficient)
model_Efficient._name='Efficient'
model_Xception =tf.keras.models.load_model(weights_Xception)
model_Xception._name = 'Xception'
model_Inception =tf.keras.models.load_model(weights_Inception)
model_Inception._name = 'Inception'
model_ResNet= tf.keras.models.load_model(weights_ResNet)
model_ResNet._name = 'ResNet'
model_MobileNetV2= tf.keras.models.load_model(weights_MobileNetV2)
model_MobileNetV2._name = 'Mobile'

In [ ]:
models = [model_Efficient, model_Xception, model_Inception, model_ResNet, model_MobileNetV2]
for i in models:
    i.trainable= False

In [ ]:
initializer = tf.keras.initializers.RandomUniform(minval=0.2, maxval=0.2)

In [ ]:
outs = [i(model_input) for i in models]
concatenated = tf.keras.layers.Dense(1, activation='relu', kernel_initializer=initializer)(concatenated)

#merged = keras.layers.Add()(outs)
Ensemble = Model(model_input, concatenated)

In [ ]:
Ensemble._name = 'Ensemble'
Ensemble.compile(Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])
Ensemble.summary()

In [ ]:
Ensemble.layers[-1].get_weights()

In [ ]:
plot_model(Ensemble, to_file='Ensemble.png', show_shapes=True, show_layer_names=True)

In [ ]:
# get the data generators
train_gen, val_gen = get_pcam_generators(r'C:\Users\20192653\Documents\8P361 - Project imaging\8p361-project-imaging-master\8p361-project-imaging-master\data')

#### saving the model

In [ ]:
ensemble_name = 'Ensemble_model'
ensemble_filepath = ensemble_name + '.json'
ensemble_weights_filepath = ensemble_name + '_weights.hdf5'

ensemble_json = Ensemble.to_json() # serialize model to JSON
with open(ensemble_filepath, 'w') as json_file:
    json_file.write(ensemble_json)
    
# define the model checkpoint and Tensorboard callbacks
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10, verbose=1), tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=1)]
#callbacks = [tf.keras.callbacks.EarlyStopping(patience=10, verbose=1), tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=1)]
checkpoint = ModelCheckpoint(ensemble_weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', ensemble_name))
callbacks_list = [checkpoint, tensorboard] + callbacks

#### Training the model

In [ ]:
train_steps = train_gen.n//train_gen.batch_size//20
val_steps = val_gen.n//val_gen.batch_size//20

history = Ensemble.fit(train_gen, steps_per_epoch=train_steps,
                    validation_steps=val_steps,validation_data=val_gen,
                    epochs=20,
                    callbacks=callbacks_list)


In [ ]:
Ensemble.layers[-1].get_weights()